In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('RealEstateAU_1000_Samples.csv')
df= pd.DataFrame(data)
df.head(5)

,index,TID,breadcrumb,category_name,property_type,building_size,land_size,preferred_size,open_date,listing_agency,...,state,zip_code,phone,latitude,longitude,product_depth,bedroom_count,bathroom_count,parking_count,RunDate
0,0,1350988,Buy>NT>DARWIN CITY,Real Estate & Property for sale in DARWIN CITY...,House,NaN,NaN,NaN,Added 2 hours ago,Professionals - DARWIN CITY,...,NT,800,08 8941 8289,NaN,NaN,premiere,2.0,1.0,1.0,2022-05-27 15:54:05
1,1,1350989,Buy>NT>DARWIN CITY,Real Estate & Property for sale in DARWIN CITY...,Apartment,171m²,NaN,171m²,Added 7 hours ago,Nick Mousellis Real Estate - Eview Group Member,...,NT,800,0411724000,NaN,NaN,premiere,3.0,2.0,2.0,2022-05-27 15:54:05
2,2,1350990,Buy>NT>DARWIN CITY,Real Estate & Property for sale in DARWIN CITY...,Unit,NaN,NaN,NaN,Added 22 hours ago,Habitat Real Estate - THE GARDENS,...,NT,800,08 8981 0080,NaN,NaN,premiere,2.0,1.0,1.0,2022-05-27 15:54:05
3,3,1350991,Buy>NT>DARWIN CITY,Real Estate & Property for sale in DARWIN CITY...,House,NaN,NaN,NaN,Added yesterday,Ray White - NIGHTCLIFF,...,NT,800,08 8982 2403,NaN,NaN,premiere,1.0,1.0,0.0,2022-05-27 15:54:05
4,4,1350992,Buy>NT>DARWIN CITY,Real Estate & Property for sale in DARWIN CITY...,Unit,201m²,NaN,201m²,Added yesterday,Carol Need Real Estate - Fannie Bay,...,NT,800,0418885966,NaN,NaN,premiere,3.0,2.0,2.0,2022-05-27 15:54:05


In [3]:
df.shape

(1000, 27)

In [4]:
df.columns

Index(['index', 'TID', 'breadcrumb', 'category_name', 'property_type',
       'building_size', 'land_size', 'preferred_size', 'open_date',
       'listing_agency', 'price', 'location_number', 'location_type',
       'location_name', 'address', 'address_1', 'city', 'state', 'zip_code',
       'phone', 'latitude', 'longitude', 'product_depth', 'bedroom_count',
       'bathroom_count', 'parking_count', 'RunDate'],
      dtype='object')

In [5]:
#Dropping irrelevant columns
df=df[[#'index', 'TID', 'breadcrumb','category_name', 
    'property_type',
       #'building_size', 'land_size', 'preferred_size', 'open_date',
       'listing_agency', 'price', 'location_number', #'location_type',
       'location_name', 
        #'address', 'address_1', 
        'city', 'state', 'zip_code',
        #'phone', 
       'latitude', 'longitude', 'product_depth', 'bedroom_count',
       'bathroom_count', 'parking_count', 'RunDate']]
df.head()

,property_type,listing_agency,price,location_number,location_name,city,state,zip_code,latitude,longitude,product_depth,bedroom_count,bathroom_count,parking_count,RunDate
0,House,Professionals - DARWIN CITY,"$435,000",139468611,"$435,000",Darwin City,NT,800,NaN,NaN,premiere,2.0,1.0,1.0,2022-05-27 15:54:05
1,Apartment,Nick Mousellis Real Estate - Eview Group Member,"Offers Over $320,000",139463755,"Offers Over $320,000",Darwin City,NT,800,NaN,NaN,premiere,3.0,2.0,2.0,2022-05-27 15:54:05
2,Unit,Habitat Real Estate - THE GARDENS,"$310,000",139462495,"$310,000",Darwin City,NT,800,NaN,NaN,premiere,2.0,1.0,1.0,2022-05-27 15:54:05
3,House,Ray White - NIGHTCLIFF,"$259,000",139451679,"$259,000",Darwin City,NT,800,NaN,NaN,premiere,1.0,1.0,0.0,2022-05-27 15:54:05
4,Unit,Carol Need Real Estate - Fannie Bay,"$439,000",139433803,"$439,000",Darwin City,NT,800,NaN,NaN,premiere,3.0,2.0,2.0,2022-05-27 15:54:05


In [6]:
#Checking Missing Values
df.isna().sum()

property_type         0
listing_agency        0
price                 0
location_number       0
location_name         0
city                  0
state                 0
zip_code              0
latitude           1000
longitude          1000
product_depth         0
bedroom_count        33
bathroom_count       33
parking_count        33
RunDate               0
dtype: int64

- step   : Check the number of missing values(NaaN)in each columns using .isna().sum()
- result : We indentified that latitude and longitude columns have the maximum mising values. 
- action : Drop latitude and longitude columns

In [7]:
#Dropping latitude and longitude columns
df_1= df.drop(['latitude','longitude'], axis=1)
df_1.isna().sum()

property_type       0
listing_agency      0
price               0
location_number     0
location_name       0
city                0
state               0
zip_code            0
product_depth       0
bedroom_count      33
bathroom_count     33
parking_count      33
RunDate             0
dtype: int64

In [8]:
#Finding pairs of columns with more than 90% similarity
duplicate_columns =[]
threshold = 0.9* len(df_1)

for col_1 in df_1.columns:
    for col_2 in df_1.columns:
        if col_1 != col_2 and (col_2 ,col_1 ) not in duplicate_columns:
            similarity_count = (df_1[col_1] == df_1[col_2]).sum()
            if similarity_count > threshold:
                duplicate_columns.append((col_1 ,col_2 ))

duplicate_columns                 

[('price', 'location_name')]

- Step   : Identify columns with more than 90% similarity, which can be a sign of data duplication. That will help in detecting redundancy and making subsewuent analyses more efficient and interpretable.
- Result : price & location_name columns have more than 90% of similarity. This finding is surpricing as price and location name are not typlically expected to be similar.
- Action : Dropping location_name column to eliminate redundancy in dataset.

In [11]:
#Droppoing location_name column
df_2 = df_1.drop(columns=['location_name'])
df_2.columns

Index(['property_type', 'listing_agency', 'price', 'location_number', 'city',
       'state', 'zip_code', 'product_depth', 'bedroom_count', 'bathroom_count',
       'parking_count', 'RunDate'],
      dtype='object')

In [13]:
#Check if all the values in each column are the same

columns_with_same_values = []

for col in df_2.columns:
    first_value = df_2[col].loc[0]
    
    if df_2[col].eq(first_value).all():
        columns_with_same_values.append(col)

columns_with_same_values

['state', 'RunDate']

- Step   : Check if all values in each columns are the same.
- Result : state & RunDate columns have the same values in each rows. Constant columns don't contribute to statistical analysis or models, as they don't provide any variation in observation.
- Action : Drop state & RunDate columns

In [15]:
#Drop state & RunDate columns
df_3 = df_2.drop(columns=['state', 'RunDate'])
df_3.columns

Index(['property_type', 'listing_agency', 'price', 'location_number', 'city',
       'zip_code', 'product_depth', 'bedroom_count', 'bathroom_count',
       'parking_count'],
      dtype='object')

In [16]:
df_3.isna().sum()

property_type       0
listing_agency      0
price               0
location_number     0
city                0
zip_code            0
product_depth       0
bedroom_count      33
bathroom_count     33
parking_count      33
dtype: int64

In [17]:
df_3.shape

(1000, 10)

- Step : Check for any further Missing values : 
- Result :bedroom_count, bathroom_count and parking_count have 33 missing values
- Action : Drop missing values 

In [40]:
df_4 = df_3.dropna()
df_4.isna().sum()

property_type      0
listing_agency     0
price              0
location_number    0
city               0
zip_code           0
product_depth      0
bedroom_count      0
bathroom_count     0
parking_count      0
dtype: int64

In [41]:
df_4

,property_type,listing_agency,price,location_number,city,zip_code,product_depth,bedroom_count,bathroom_count,parking_count
0,House,Professionals - DARWIN CITY,"$435,000",139468611,Darwin City,800,premiere,2.0,1.0,1.0
1,Apartment,Nick Mousellis Real Estate - Eview Group Member,"Offers Over $320,000",139463755,Darwin City,800,premiere,3.0,2.0,2.0
2,Unit,Habitat Real Estate - THE GARDENS,"$310,000",139462495,Darwin City,800,premiere,2.0,1.0,1.0
3,House,Ray White - NIGHTCLIFF,"$259,000",139451679,Darwin City,800,premiere,1.0,1.0,0.0
4,Unit,Carol Need Real Estate - Fannie Bay,"$439,000",139433803,Darwin City,800,premiere,3.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
995,House,United Realty NT - Parap,2 Residence,138346247,Virginia,834,feature,4.0,3.0,6.0
996,House,Kassiou Constructions - HOWARD SPRINGS,"$601,000",138333062,Humpty Doo,836,standard,4.0,2.0,2.0
997,House,Kassiou Constructions - HOWARD SPRINGS,"$655,000",138333058,Humpty Doo,836,standard,4.0,2.0,2.0
998,House,Kassiou Constructions - HOWARD SPRINGS,"$675,000",138333050,Lee Point,810,standard,4.0,2.0,3.0


In [42]:
#Extract Price from the text (ex :Offers Over $320,000 )
def extract_price(price_text):
    
    #Using regex to find a pattern that matches the price and ignores any following text
    price_match = re.search(r'\$[\d, ]*(?:\.\d{2})?' , str(price_text))
    
    if price_match:
        
        #remove spaces from the captured price and return
        return price_match.group(0).replace('','').replace('','')
    else:
        return None

#Applying the function to the price column in dataset
df_4['price'] = df_4['price'].apply(extract_price)

df_4.head()

,property_type,listing_agency,price,location_number,city,zip_code,product_depth,bedroom_count,bathroom_count,parking_count
0,House,Professionals - DARWIN CITY,"$435,000",139468611,Darwin City,800,premiere,2.0,1.0,1.0
1,Apartment,Nick Mousellis Real Estate - Eview Group Member,"$320,000",139463755,Darwin City,800,premiere,3.0,2.0,2.0
2,Unit,Habitat Real Estate - THE GARDENS,"$310,000",139462495,Darwin City,800,premiere,2.0,1.0,1.0
3,House,Ray White - NIGHTCLIFF,"$259,000",139451679,Darwin City,800,premiere,1.0,1.0,0.0
4,Unit,Carol Need Real Estate - Fannie Bay,"$439,000",139433803,Darwin City,800,premiere,3.0,2.0,2.0


- Step : Remove aditional text from the price value ( ex.Offers Over $320,000) . Design a function to Return the price value in $100,000 format
- Result : The function uses a redular expression(regex) to extract the numeric price value, including the $ symbol.
- Action : Transformed the price values into a consistent formate that can be used for numerical analysis.

In [43]:
df_4.isna().sum()

property_type        0
listing_agency       0
price              223
location_number      0
city                 0
zip_code             0
product_depth        0
bedroom_count        0
bathroom_count       0
parking_count        0
dtype: int64

In [44]:
df_5 = df_4.dropna()
df_5.isna().sum()

property_type      0
listing_agency     0
price              0
location_number    0
city               0
zip_code           0
product_depth      0
bedroom_count      0
bathroom_count     0
parking_count      0
dtype: int64

In [46]:
df_5.head(2)

,property_type,listing_agency,price,location_number,city,zip_code,product_depth,bedroom_count,bathroom_count,parking_count
0,House,Professionals - DARWIN CITY,"$435,000",139468611,Darwin City,800,premiere,2.0,1.0,1.0
1,Apartment,Nick Mousellis Real Estate - Eview Group Member,"$320,000",139463755,Darwin City,800,premiere,3.0,2.0,2.0


In [50]:
#Remove currency symbol and 
# convert price column into a integer datatype
df_5_copy = df_5.copy()

df_5_copy['price'] = df_5_copy['price'].replace('[\$, .]', '' , regex=True).astype(int)
df_6 = df_5_copy
df_6 = df_6.reset_index(drop=True)
df_6

,property_type,listing_agency,price,location_number,city,zip_code,product_depth,bedroom_count,bathroom_count,parking_count
0,House,Professionals - DARWIN CITY,435000,139468611,Darwin City,800,premiere,2.0,1.0,1.0
1,Apartment,Nick Mousellis Real Estate - Eview Group Member,320000,139463755,Darwin City,800,premiere,3.0,2.0,2.0
2,Unit,Habitat Real Estate - THE GARDENS,310000,139462495,Darwin City,800,premiere,2.0,1.0,1.0
3,House,Ray White - NIGHTCLIFF,259000,139451679,Darwin City,800,premiere,1.0,1.0,0.0
4,Unit,Carol Need Real Estate - Fannie Bay,439000,139433803,Darwin City,800,premiere,3.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
739,House,Real Estate Central - DARWIN CITY,495000,138350855,Woodroffe,830,feature,4.0,2.0,2.0
740,House,Kassiou Constructions - HOWARD SPRINGS,601000,138333062,Humpty Doo,836,standard,4.0,2.0,2.0
741,House,Kassiou Constructions - HOWARD SPRINGS,655000,138333058,Humpty Doo,836,standard,4.0,2.0,2.0
742,House,Kassiou Constructions - HOWARD SPRINGS,675000,138333050,Lee Point,810,standard,4.0,2.0,3.0


In [52]:
df_6.dtypes

property_type       object
listing_agency      object
price                int32
location_number      int64
city                object
zip_code             int64
product_depth       object
bedroom_count      float64
bathroom_count     float64
parking_count      float64
dtype: object

In [57]:
#Cehck if the DataSet contains any Negative values # Select only numeric columns from above
negative_count = df_6[['price','location_number','zip_code','bedroom_count','bathroom_count','parking_count' ]]\
                        .apply(lambda x: (x<0).sum())
negative_count

price              0
location_number    0
zip_code           0
bedroom_count      0
bathroom_count     0
parking_count      0
dtype: int64

- Step   : Cehck if there are any Negetive values in the specified numeric columns. As -ve values i nthe dataset might not be logical sense. By removing -ve values , tt helps to identify potential error or inconsistences in the data.

- Result : There is not -ve values in the dataset.

In [58]:
cleaned_df = df_6
cleaned_df

,property_type,listing_agency,price,location_number,city,zip_code,product_depth,bedroom_count,bathroom_count,parking_count
0,House,Professionals - DARWIN CITY,435000,139468611,Darwin City,800,premiere,2.0,1.0,1.0
1,Apartment,Nick Mousellis Real Estate - Eview Group Member,320000,139463755,Darwin City,800,premiere,3.0,2.0,2.0
2,Unit,Habitat Real Estate - THE GARDENS,310000,139462495,Darwin City,800,premiere,2.0,1.0,1.0
3,House,Ray White - NIGHTCLIFF,259000,139451679,Darwin City,800,premiere,1.0,1.0,0.0
4,Unit,Carol Need Real Estate - Fannie Bay,439000,139433803,Darwin City,800,premiere,3.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
739,House,Real Estate Central - DARWIN CITY,495000,138350855,Woodroffe,830,feature,4.0,2.0,2.0
740,House,Kassiou Constructions - HOWARD SPRINGS,601000,138333062,Humpty Doo,836,standard,4.0,2.0,2.0
741,House,Kassiou Constructions - HOWARD SPRINGS,655000,138333058,Humpty Doo,836,standard,4.0,2.0,2.0
742,House,Kassiou Constructions - HOWARD SPRINGS,675000,138333050,Lee Point,810,standard,4.0,2.0,3.0


The dataset is cleaned and ready for analysis and exploration.